In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Amazonas - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_AM.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego,Amazonas - Consumo de Cimento (t)
0,2003,623.227503,0.695898,3.993873,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,4.754469e+07,2.519796e+06,12.490621,4.318518e+07,8.630942,418.347000
1,2004,631.482259,0.697632,4.187699,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,4.823122e+07,2.538092e+06,12.509135,4.338144e+07,8.558110,587.382000
2,2005,637.838656,0.699241,4.449583,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,4.883625e+07,2.540251e+06,12.511306,4.349664e+07,8.526727,613.286000
3,2006,664.588528,0.700728,4.788581,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,4.938283e+07,2.557702e+06,12.511871,4.357888e+07,8.446276,626.147000
4,2007,691.804806,0.702099,5.207764,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,4.980707e+07,2.573779e+06,12.484991,4.354651e+07,8.434407,666.931000
5,2008,694.929186,0.703356,5.596891,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,8.396886,769.233000
6,2009,715.226516,0.704503,5.981096,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,8.420659,744.252000
7,2010,753.125620,0.705545,6.348653,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,8.389077,796.116000
8,2011,768.056058,0.706485,6.656378,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,8.376520,948.011000
9,2012,750.564710,0.707029,6.998333,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,11.321769,994.932000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
0,623.227503,0.695898,3.993873,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,4.754469e+07,2.519796e+06,12.490621,4.318518e+07,8.630942
1,631.482259,0.697632,4.187699,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,4.823122e+07,2.538092e+06,12.509135,4.338144e+07,8.558110
2,637.838656,0.699241,4.449583,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,4.883625e+07,2.540251e+06,12.511306,4.349664e+07,8.526727
3,664.588528,0.700728,4.788581,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,4.938283e+07,2.557702e+06,12.511871,4.357888e+07,8.446276
4,691.804806,0.702099,5.207764,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,4.980707e+07,2.573779e+06,12.484991,4.354651e+07,8.434407
5,694.929186,0.703356,5.596891,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,8.396886
6,715.226516,0.704503,5.981096,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,8.420659
7,753.125620,0.705545,6.348653,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,8.389077
8,768.056058,0.706485,6.656378,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,8.376520
9,750.564710,0.707029,6.998333,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,11.321769


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      587.382000
1      613.286000
2      626.147000
3      666.931000
4      769.233000
5      744.252000
6      796.116000
7      948.011000
8      994.932000
9     1036.606000
10     771.567857
11     746.867486
12     739.457657
13     687.211000
14     350.177000
15     533.444000
16     595.320000
17     610.155000
18     640.274000
19            NaN
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
0,-0.064363,-1.902692,-1.871291,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.657429,0.147053,1.022575,-1.837871,-0.844534
1,0.015089,-1.569079,-1.691623,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.420136,0.502943,1.329420,-1.291529,-0.879556
2,0.076268,-1.259532,-1.448868,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.211011,0.544940,1.365402,-0.970828,-0.894648
3,0.333733,-0.973305,-1.134632,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.022092,0.884383,1.374762,-0.741870,-0.933333
4,0.595687,-0.709650,-0.746067,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.875457,1.197126,0.929263,-0.831994,-0.939040
5,0.625759,-0.467820,-0.385363,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.749223,1.100359,0.526764,-0.973198,-0.957083
6,0.821119,-0.247067,-0.029222,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.644904,1.219506,0.082911,-1.085772,-0.945651
7,1.185894,-0.046644,0.311488,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.223579,1.104278,0.478356,-0.139598,-0.960838
8,1.329598,0.134197,0.596735,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.133776,0.656315,0.531211,0.398719,-0.966876
9,1.161246,0.238977,0.913712,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.413989,0.240530,-0.232601,0.529576,0.449384


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      587.382000
1      613.286000
2      626.147000
3      666.931000
4      769.233000
5      744.252000
6      796.116000
7      948.011000
8      994.932000
9     1036.606000
10     771.567857
11     746.867486
12     739.457657
13     687.211000
14     350.177000
15     533.444000
16     595.320000
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
17,-1.367217,0.772056,1.730097,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.870475,-0.673546,-0.73585,0.899019,0.934453


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    610.155
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1825607731, 2892395165, 2855325632, 2254990615, 157274270, 428297130, 418453928, 1593439075, 4090650169, 348384090, 3771787104, 2488197172, 1564563104, 3153213941, 2677833731, 1831215, 1546656009, 1375169617, 2465227133, 954144163, 852964046, 1515428709, 1847809529, 489096961, 1874935200, 1015873681, 101527352, 1876415504, 2668815027, 179003062, 2738713207, 257818472, 2917848708, 603643384, 1307376797, 1177758344, 430627637, 3295092807, 3494939199, 466712748, 3577041577, 2827876322, 567568987, 725775848, 1536973846, 213165498, 3729222273, 1020016431, 163715390, 3934841988, 76990011, 1217709339, 2081263677, 117101917, 3130213660, 555315591, 1566756813, 261486761, 672943081, 2530713524, 3390400373, 2841957401, 3364374965, 865158403, 1926813179, 711316826, 1685796548, 1785661232, 1265828254, 1090101564, 1140811799, 3308156945, 3236415826, 3554510252, 3268440174, 1364958078, 478114496, 2881398462, 2550464290, 3014505698, 870138790, 1066713949, 3447127541, 3716613598, 3512876999, 397451453

val_loss: 22160.509765625


Step: 87 ___________________________________________
val_loss: 10865.5849609375


Step: 88 ___________________________________________
val_loss: 11105.8310546875


Step: 89 ___________________________________________
val_loss: 16511.166015625


Step: 90 ___________________________________________
val_loss: 11045.30859375


Step: 91 ___________________________________________
val_loss: 44890.5


Step: 92 ___________________________________________
val_loss: 10836.2158203125


Step: 93 ___________________________________________
val_loss: 66194.4140625


Step: 94 ___________________________________________
val_loss: 22146.201171875


Step: 95 ___________________________________________
val_loss: 14386.37109375


Step: 96 ___________________________________________
val_loss: 10838.8203125


Step: 97 ___________________________________________
val_loss: 29494.310546875


Step: 98 ___________________________________________
val_loss: 10834.6240234375


Step: 99 _

In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 462ms/step - loss: 543217.1250 - val_loss: 239657.5781
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 518515.3438 - val_loss: 59241.1445
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 518201.0000 - val_loss: 394774.4688
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 529884.3750 - val_loss: 437295.1250
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 444307.0000 - val_loss: 424712.3438
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 434925.7812 - val_loss: 443273.1250
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 435354.5312 - val_loss: 418768.2500
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 420525.6875 - val_loss: 403043.0938
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 374405.8750 - val_loss: 495463.5000
Epoch 10/1

1/1 [==============================] - 0s 15ms/step - loss: 84275.2266 - val_loss: 29021.7500
Epoch 76/10000
1/1 [==============================] - 0s 15ms/step - loss: 88378.1172 - val_loss: 35526.4570
Epoch 77/10000
1/1 [==============================] - 0s 15ms/step - loss: 86509.3438 - val_loss: 32087.6797
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 82655.1484 - val_loss: 189659.6719
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 94064.9297 - val_loss: 59380.5430
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 91930.9062 - val_loss: 54589.3945
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 96200.3672 - val_loss: 64379.2500
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 83586.3203 - val_loss: 60990.8242
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 88870.5234 - val_loss: 26192.8066
Epoch 84/10000
1/1 [=============

Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 40844.4375 - val_loss: 25602.1543
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 42308.5664 - val_loss: 29702.5625
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 40483.2305 - val_loss: 25864.6309
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 36255.9844 - val_loss: 21290.7832
Epoch 154/10000
1/1 [==============================] - 0s 15ms/step - loss: 36571.6992 - val_loss: 21757.8848
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 41928.0195 - val_loss: 23176.6387
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 41284.1211 - val_loss: 17394.7969
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 46052.2422 - val_loss: 19651.6328
Epoch 158/10000
1/1 [==============================] - 0s 15ms/step - loss: 35209.7617 - val_loss: 20102.3496
Epoch 159/

1/1 [==============================] - 0s 13ms/step - loss: 17934.5410 - val_loss: 11051.4854
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 21401.9004 - val_loss: 11031.4541
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 20730.7676 - val_loss: 11298.6875
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 19017.7344 - val_loss: 10905.1533
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 19489.2676 - val_loss: 15863.3086
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 24988.1035 - val_loss: 10947.6572
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 21396.0195 - val_loss: 11595.3564
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 21998.8203 - val_loss: 11232.0420
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 22467.2168 - val_loss: 11077.1172
Epoch 233/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 13565.0225 - val_loss: 15586.1768
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 10372.8916 - val_loss: 17257.2715
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 10656.9541 - val_loss: 14431.9639
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 11130.5059 - val_loss: 16437.0957
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 10948.8525 - val_loss: 16174.4814
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 17861.4395 - val_loss: 14181.7314
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 11568.7480 - val_loss: 15721.1328
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 10641.5713 - val_loss: 16701.5000
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 11116.0986 - val_loss: 15765.0234
Epoch 307/10000
1/1 [=====

...

1/1 [==============================] - 0s 14ms/step - loss: 15039.7275 - val_loss: 37796.6914
Epoch 672/10000
1/1 [==============================] - 0s 14ms/step - loss: 14753.6797 - val_loss: 37098.1719
Epoch 673/10000
1/1 [==============================] - 0s 14ms/step - loss: 13371.6143 - val_loss: 40347.2930
Epoch 674/10000
1/1 [==============================] - 0s 13ms/step - loss: 14934.0264 - val_loss: 36808.6758
Epoch 675/10000
1/1 [==============================] - 0s 13ms/step - loss: 14078.7334 - val_loss: 35303.4180
Epoch 676/10000
1/1 [==============================] - 0s 14ms/step - loss: 12949.7031 - val_loss: 38077.0547
Epoch 677/10000
1/1 [==============================] - 0s 14ms/step - loss: 14505.6973 - val_loss: 36131.9336
Epoch 678/10000
1/1 [==============================] - 0s 13ms/step - loss: 15069.8799 - val_loss: 39354.3867
Epoch 679/10000
1/1 [==============================] - 0s 13ms/step - loss: 13466.0703 - val_loss: 39054.0586
Epoch 680/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 20174.7656 - val_loss: 36674.4258
Epoch 746/10000
1/1 [==============================] - 0s 14ms/step - loss: 19620.9434 - val_loss: 36334.0195
Epoch 747/10000
1/1 [==============================] - 0s 14ms/step - loss: 19541.0996 - val_loss: 37373.8320
Epoch 748/10000
1/1 [==============================] - 0s 13ms/step - loss: 19232.3770 - val_loss: 37742.7617
Epoch 749/10000
1/1 [==============================] - 0s 14ms/step - loss: 19455.2051 - val_loss: 37533.1406
Epoch 750/10000
1/1 [==============================] - 0s 14ms/step - loss: 19412.0039 - val_loss: 37789.6992
Epoch 751/10000
1/1 [==============================] - 0s 14ms/step - loss: 19191.3281 - val_loss: 38594.2539
Epoch 752/10000
1/1 [==============================] - 0s 14ms/step - loss: 18910.1895 - val_loss: 39074.8164
Epoch 753/10000
1/1 [==============================] - 0s 14ms/step - loss: 18929.2910 - val_loss: 38817.4414
Epoch 754/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 16726.7246 - val_loss: 38300.8164
Epoch 820/10000
1/1 [==============================] - 0s 14ms/step - loss: 16817.6406 - val_loss: 40053.3047
Epoch 821/10000
1/1 [==============================] - 0s 14ms/step - loss: 16671.0039 - val_loss: 51466.6523
Epoch 822/10000
1/1 [==============================] - 0s 14ms/step - loss: 18229.1660 - val_loss: 43823.0156
Epoch 823/10000
1/1 [==============================] - 0s 14ms/step - loss: 17968.8809 - val_loss: 45246.5938
Epoch 824/10000
1/1 [==============================] - 0s 14ms/step - loss: 17953.0215 - val_loss: 44985.5469
Epoch 825/10000
1/1 [==============================] - 0s 14ms/step - loss: 17911.5625 - val_loss: 45984.7812
Epoch 826/10000
1/1 [==============================] - 0s 14ms/step - loss: 17833.6992 - val_loss: 44222.4805
Epoch 827/10000
1/1 [==============================] - 0s 14ms/step - loss: 17941.6602 - val_loss: 45299.3555
Epoch 828/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 18330.1348 - val_loss: 43814.1719
Epoch 894/10000
1/1 [==============================] - 0s 14ms/step - loss: 18270.1191 - val_loss: 44197.4570
Epoch 895/10000
1/1 [==============================] - 0s 14ms/step - loss: 18424.8047 - val_loss: 44413.8242
Epoch 896/10000
1/1 [==============================] - 0s 14ms/step - loss: 18422.4355 - val_loss: 44372.8750
Epoch 897/10000
1/1 [==============================] - 0s 13ms/step - loss: 18354.6465 - val_loss: 44948.4492
Epoch 898/10000
1/1 [==============================] - 0s 13ms/step - loss: 18328.6914 - val_loss: 44986.9023
Epoch 899/10000
1/1 [==============================] - 0s 13ms/step - loss: 18255.1543 - val_loss: 44222.2539
Epoch 900/10000
1/1 [==============================] - 0s 13ms/step - loss: 18414.1855 - val_loss: 44310.3086
Epoch 901/10000
1/1 [==============================] - 0s 14ms/step - loss: 18328.7910 - val_loss: 43828.6445
Epoch 902/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 18219.0137 - val_loss: 45542.9688
Epoch 968/10000
1/1 [==============================] - 0s 14ms/step - loss: 18200.3301 - val_loss: 45278.4648
Epoch 969/10000
1/1 [==============================] - 0s 14ms/step - loss: 18221.2422 - val_loss: 45557.3438
Epoch 970/10000
1/1 [==============================] - 0s 14ms/step - loss: 18250.4492 - val_loss: 45140.1367
Epoch 971/10000
1/1 [==============================] - 0s 14ms/step - loss: 18264.7852 - val_loss: 45146.8477
Epoch 972/10000
1/1 [==============================] - 0s 14ms/step - loss: 18250.9199 - val_loss: 45281.1836
Epoch 973/10000
1/1 [==============================] - 0s 14ms/step - loss: 18355.1797 - val_loss: 45649.1133
Epoch 974/10000
1/1 [==============================] - 0s 14ms/step - loss: 18351.1270 - val_loss: 45544.1758
Epoch 975/10000
1/1 [==============================] - 0s 14ms/step - loss: 18241.4277 - val_loss: 45012.9805
Epoch 976/10000
1/1 [=====

In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[610.155](test_target) - [[609.18494]](prediction) = 0.9700634765624727


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
0,-1.254290,-1.651948,-1.338730,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.598377,-1.869038,0.296993,-1.892022,1.914585
1,-1.060035,-1.103970,-1.097441,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.042201,-0.905748,0.977617,-0.681310,1.018176
2,-0.910453,-0.595523,-0.771428,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.552049,-0.792075,1.057430,0.029372,0.631905
3,-0.280963,-0.125379,-0.349420,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.109253,0.126701,1.078190,0.536750,-0.358282
4,0.359504,0.307689,0.172411,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.234434,0.973204,0.090015,0.337034,-0.504360
5,0.433028,0.704909,0.656824,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.530305,0.711284,-0.802782,0.024121,-0.966173
6,0.910675,1.067508,1.135111,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.774812,1.033781,-1.787306,-0.225348,-0.673571
7,1.802535,1.396714,1.592673,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.762328,0.721891,-0.910157,1.871404,-1.062282


train_target:


,Amazonas - Consumo de Cimento (t)
0,587.382
1,613.286
2,626.147
3,666.931
4,769.233
5,744.252
6,796.116
7,948.011


test_input:


...

,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
17,-1.367217,0.772056,1.730097,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.870475,-0.673546,-0.73585,0.899019,0.934453


test_target:


,Amazonas - Consumo de Cimento (t)
17,610.155


1/1 [==============================] - 0s 34ms/step
Error: 0.9700634765624727




[823.811279296875,
 888.076904296875,
 999.6116943359375,
 904.079345703125,
 778.4517822265625,
 763.470703125,
 715.065673828125,
 647.9451904296875,
 523.9060668945312,
 609.1849365234375]

In [23]:
display(targets)
display(predictions)

[994.932,
 1036.606,
 771.5678571428572,
 746.8674857142857,
 739.4576571428571,
 687.211,
 350.177,
 533.444,
 595.32,
 610.155]

[823.811279296875,
 888.076904296875,
 999.6116943359375,
 904.079345703125,
 778.4517822265625,
 763.470703125,
 715.065673828125,
 647.9451904296875,
 523.9060668945312,
 609.1849365234375]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

137.1933202636719

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.1941670074147554